In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import sys
import json
from dotenv import load_dotenv
import pandas as pd
import datetime
import pytz
from dateutil.relativedelta import relativedelta

# Dataset Setup 

In [8]:
catalog_df = pd.read_csv('tracks_features.csv')
catalog_df.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


# Connecting to Spotify API

In [2]:
load_dotenv()
scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [3]:
playlists_res = sp.current_user_playlists()['items']
playlists = {}

for item in playlists_res:
  playlists[item['name']] = item['id']

playlists
id = playlists['RO tation'] # This is what the input to the model will be, playlist ID

# Constructing Input Data From Playlist

In [4]:
def tracks_from_playlist(id):
    """
    Given a playlist id, returns a pandas dataframe consisting of key elements of each song
    """
    playlist = sp.playlist(id)
    tracks = []
    for item in playlist['tracks']['items']:
        if item['track']['id'] is not None:
            track = item['track']
            track_id = track['id']
            artist_ids = [artist['id'] for artist in track['artists']] if len(track['artists']) > 0 else None
            artist_names = [artist['name'] for artist in track['artists']] if len(track['artists']) > 0 else None
            track_info = {
                'track_name': track['name'],
                'track_id': track_id,
                'artists': artist_names,
                'artist_ids': artist_ids,
                'album_name': track['album']['name'],
                'duration_ms': track['duration_ms'],
                'explicit': track['explicit'],
                'popularity': track['popularity'],
                'date_added': item['added_at']
            }
            tracks.append(track_info)

    tracks_df = pd.DataFrame(tracks)
    tracks_df['date_added'] = pd.to_datetime(tracks_df['date_added'], utc=True)
    now = datetime.datetime.now(pytz.utc)
    tracks_df['months_since_added'] = tracks_df['date_added'].apply(lambda x: relativedelta(now, x).months)
    tracks_df = tracks_df.drop(['date_added'], axis=1)

    return tracks_df

In [5]:
def extract_tracks_features(ids):
  """
  Given a list of track ids, returns a pandas dataframe of key audio features of each track
  """
  audio_features_list = []
  for track_id in ids:
    if track_id is not None:
      audio_features = sp.audio_features(track_id)[0]
      audio_features_list.append(audio_features)

  # convert list of dictionaries to Pandas DataFrame
  audio_features_df = pd.DataFrame.from_records(audio_features_list, columns=audio_features_list[0].keys())

  # add track ID column to DataFrame
  audio_features_df['track_id'] = ids

  # re-order columns to put track_id first
  cols = audio_features_df.columns.tolist()
  cols = cols[-1:] + cols[:-1]
  audio_features_df = audio_features_df[cols]
  audio_features_df = audio_features_df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'track_id'], axis=1)
  return audio_features_df

In [6]:
def create_playlist_df(id):
    """
    Returns a dataframe for the playlsit with the given id. Attributes include thoe returned by
    Spotify Web API's GET Audio Features endpoint and track meta data like name, artist, etc.
    """
    df1 = tracks_from_playlist(id)
    df2 = extract_tracks_features(df1['track_id'])
    playlist_df = pd.concat([df1,df2], axis=1)
    return playlist_df

In [7]:
playlist_df = create_playlist_df(id)
playlist_df

,track_name,track_id,artists,artist_ids,album_name,duration_ms,explicit,popularity,months_since_added,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Favorite Song,1sFstGV1Z3Aw5TDFCiT7vK,"[Chance the Rapper, Childish Gambino]","[1anyVhU62p31KFi8MEzkbf, 73sIBHcqh3Z3NyqHKZ7FOL]",Acid Rap,185062,True,63,3,0.725,...,-9.030,1,0.377,0.0321,0.000000,0.0504,0.811,155.918,185062,4
1,No Problem (feat. Lil Wayne & 2 Chainz),0v9Wz8o0BT8DU38R4ddjeH,"[Chance the Rapper, Lil Wayne, 2 Chainz]","[1anyVhU62p31KFi8MEzkbf, 55Aa2cqylxrFIXC767Z86...",Coloring Book,304606,True,67,3,0.652,...,-5.192,0,0.174,0.1560,0.000000,0.1230,0.788,135.018,304607,4
2,All We Got (feat. Kanye West & Chicago Childre...,3ZLyt2ndLFBh148XRYjYYZ,"[Chance the Rapper, Kanye West, Chicago Childr...","[1anyVhU62p31KFi8MEzkbf, 5K4W6rqBFWDnAN6FQUkS6...",Coloring Book,203794,False,57,3,0.531,...,-8.615,0,0.462,0.3080,0.000002,0.0835,0.398,131.921,203794,4
3,Lord Knows,1QBwk6GTCxVdC2hoSw9tlM,"[Drake, Rick Ross]","[3TVXtAsR1Inumwj472S9r4, 1sBkRIssrMs1AbVkOJbc7a]",Take Care (Deluxe),307640,True,64,3,0.417,...,-5.551,0,0.367,0.0786,0.000000,0.8340,0.303,167.579,307640,4
4,Rock N Roll (feat. Kanye West & Kid Cudi),1ZM8toCOlnfBKJdvR8GqUq,"[Pusha T, Kanye West, Kid Cudi]","[0ONHkAv9pCAFxb0zJwDNTy, 5K4W6rqBFWDnAN6FQUkS6...",It's Almost Dry,233922,True,59,3,0.687,...,-4.573,1,0.191,0.0526,0.000000,0.0718,0.359,156.141,233923,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"The Good, The Bad, The Ugly (feat. Kanye West)...",52uU5uNt9lOstGCc6C7QEJ,"[Consequence, Kanye West]","[5KkgZ31b9eDwkvo9oMIrSm, 5K4W6rqBFWDnAN6FQUkS6x]",Don't Quit Your Day Job,251906,True,50,0,0.473,...,-3.372,0,0.444,0.0271,0.000000,0.0784,0.721,94.571,251907,4
95,Herside Story,564oa00vY05d1uYnTEAAmE,"[GoldLink, Hare Squead]","[5XenQ7XfcvQdfIbpLEFaKQ, 3OqgnoaUHWX4ih4oAL0zzQ]",At What Cost,183760,True,61,0,0.885,...,-5.487,1,0.286,0.6560,0.000004,0.1050,0.754,148.034,183760,4
96,Surf (feat. Gunna),6igOR5QCn09zAVF2ikZjsV,"[Young Thug, Gunna]","[50co4Is1HCEo8bhOyUWKpn, 2hlmm7s2ICUX0LVIhVFlZQ]",So Much Fun,184093,True,55,0,0.930,...,-8.053,1,0.250,0.1010,0.000000,0.0992,0.596,140.987,184093,4
97,Bad Bad Bad (feat. Lil Baby),1GeNui6m825V8jP4uKiIaH,"[Young Thug, Lil Baby]","[50co4Is1HCEo8bhOyUWKpn, 5f7VJjfbwm532GiveGC0ZK]",So Much Fun,149040,True,68,0,0.974,...,-8.888,1,0.184,0.0976,0.000000,0.1510,0.892,111.959,149040,4


# Generate Recommendations